In [78]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [80]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [81]:
#Scale for better intepretation
data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [82]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

# OLS

In [83]:
data_2013 = data[data['Year'] == 2013]
OLS = sm.ols('np.log(Market_share) ~ Price + Range + HP + Chargetime + China', data).fit(cov_type='HC3')
OLS.summary()#.tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     np.log(Market_share)   R-squared:                       0.207
Model:                              OLS   Adj. R-squared:                  0.195
Method:                   Least Squares   F-statistic:                     21.01
Date:                  Sun, 12 May 2024   Prob (F-statistic):           3.18e-18
Time:                          11:44:01   Log-Likelihood:                -716.37
No. Observations:                   334   AIC:                             1445.
Df Residuals:                       328   BIC:                             1468.
Df Model:                             5                                         
Covariance Type:                    HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.4240      0.700     -9.171      0.000      -7.797      -5.051
Price         -0.0322      0.005     -6.537      0.000      -0.042      -0.023
Range          0.0835      0.014      5.836      0.000       0.055       0.112
HP             0.0017      0.012      0.136      0.892      -0.022       0.026
Chargetime    -0.0132      0.012     -1.058      0.290      -0.038       0.011
China         -2.2275      0.362     -6.152      0.000      -2.937      -1.518
==============================================================================
Omnibus:                       16.332   Durbin-Watson:                   0.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.595
Skew:                          -0.560   Prob(JB):                     0.000151
Kurtosis:                       3.096   Cond. No.                         450.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Willingness to pay

In [84]:
alpha = OLS.params[1] # Price coefficient
beta = list(OLS.params[:1]) + list(OLS.params[2:]) # Other coefficients
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(OLS.params.index[2:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[2:][i], beta_alpha_ratio[i])

W2P: Range 2.592680339023965
W2P: HP 0.05154449225980682
W2P: Chargetime -0.40934384664303847
W2P: China -69.14201739405368


/var/folders/ys/szn3cjzn7zg_mdp21slh2_hr0000gn/T/ipykernel_85950/608251932.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = OLS.params[1] # Price coefficient


NOTE: English delimiter\
25,926 DKK for every 10 increase in Range\
515 DKK for every 10 increase in HP\
-4,093 DKK for every 10 increase in Chargetime\
-691,420 DKK for Chinese cars OR -69,000 DKK for Chinese cars???

# Logit

In [85]:
X = data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = data['Price']

In [86]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [87]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,1.304365,0.382836,0.970142,0.156874,0.199431,0.249744,0.099531,0.114251,3.095818,...,0.103425,0.069223,0.061207,0.039744,0.213811,0.329879,0.0948,0.073535,0.105073,1.184103
U6,0.766657,1.0,0.293504,0.743766,0.120269,0.152895,0.191468,0.076306,0.087591,2.37343,...,0.079291,0.05307,0.046925,0.03047,0.16392,0.252904,0.072679,0.056376,0.080555,0.907801
e-tron,2.612085,3.407111,1.0,2.534093,0.409769,0.520932,0.652353,0.259982,0.298433,8.08654,...,0.270154,0.180815,0.159877,0.103816,0.558493,0.861673,0.247626,0.19208,0.27446,3.092979
e-tron GT,1.030777,1.344509,0.394619,1.0,0.161703,0.205569,0.257431,0.102594,0.117767,3.191099,...,0.106608,0.071353,0.06309,0.040968,0.220392,0.340032,0.097718,0.075798,0.108307,1.220547
Q4 e-tron,6.374524,8.314703,2.440397,6.184192,1.0,1.27128,1.592001,0.63446,0.728295,19.734368,...,0.659282,0.441261,0.390164,0.253352,1.362945,2.102824,0.604306,0.468751,0.66979,7.548096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,3.03141,3.954064,1.160533,2.940898,0.475551,0.604558,0.757077,0.301718,0.346341,9.384696,...,0.313522,0.209842,0.185543,0.120482,0.648149,1.0,0.287378,0.222915,0.318519,3.589504
C40,10.548495,13.759084,4.038343,10.233537,1.65479,2.1037,2.634426,1.049898,1.205175,32.656224,...,1.090973,0.730194,0.645639,0.419245,2.255386,3.479732,1.0,0.775684,1.108362,12.49051
EX30,13.598967,17.738011,5.206173,13.192927,2.133331,2.71206,3.396264,1.353513,1.553694,42.099931,...,1.406466,0.941355,0.832348,0.540484,2.907611,4.48602,1.289186,1.0,1.428884,16.102584


In [88]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, OLS)
marginal_effects

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marginal_effects.iloc[i,j] = coefficients[j]*ccp[i]*(1-ccp[i]) #dv/dz*P_i*(1-P_i)


,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-0.795428,-0.003989,0.010342,0.000206,-0.001633,-0.275805
Leaf_2013,-0.75061,-0.003764,0.009759,0.000194,-0.001541,-0.260265
Zoe_2013,-1.029828,-0.005164,0.01339,0.000266,-0.002114,-0.357081
Fortwo_2013,-0.143736,-0.000721,0.001869,0.000037,-0.000295,-0.049839
Model S_2013,-1.551645,-0.007781,0.020174,0.000401,-0.003185,-0.538015
...,...,...,...,...,...,...
up!_2023,-0.031202,-0.000156,0.000406,0.000008,-0.000064,-0.010819
C40_2023,-0.107255,-0.000538,0.001395,0.000028,-0.00022,-0.037189
EX30_2023,-0.13758,-0.00069,0.001789,0.000036,-0.000282,-0.047704


In [89]:
marginal_effects[OLS.params.index].mean()

Intercept    -0.184726
Price        -0.000926
Range         0.002402
HP            0.000048
Chargetime   -0.000379
China        -0.064051
dtype: object

In [90]:
elasticity = fun.elasticity(logit_data, OLS)
elasticity

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:102: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elasticity.iloc[i,j] = ((coefficients[j])*X.iloc[i,j]*(1 - ccp[i]))


,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-5.493923,-0.688783,2.107238,0.023716,-0.203003,-0.0
Leaf_2013,-5.556159,-0.706868,2.369504,0.021112,-0.490446,-0.0
Zoe_2013,-5.135897,-0.445963,2.437352,0.017789,-0.590409,-0.0
Fortwo_2013,-6.276908,-0.662476,1.036474,0.01298,-0.773117,-0.0
Model S_2013,-3.802935,-1.372296,3.011239,0.066354,-0.234201,-0.0
...,...,...,...,...,...,...
up!_2023,-6.392657,-0.56055,2.127798,0.013385,-0.629899,-0.0
C40_2023,-6.314904,-1.363849,3.826147,0.06562,-0.362972,-0.0
EX30_2023,-6.283352,-1.160348,3.880556,0.043528,-0.361158,-0.0


In [91]:
elasticity[OLS.params.index].mean()

Intercept    -6.212443
Price        -1.466067
Range         3.181566
HP             0.04039
Chargetime   -0.435161
China        -0.259694
dtype: object

In [92]:
print(OLS.params)

X = logit_data[OLS.params.index]
print(X[:5])

CCP = logit_data['CCP']
print(CCP[:5])


Intercept    -6.424012
Price        -0.032216
Range         0.083525
HP            0.001661
Chargetime   -0.013187
China        -2.227452
dtype: float64
   Intercept     Price  Range    HP  Chargetime  China
0        1.0  25.00000   29.5  16.7          18      0
1        1.0  25.36900   32.8  14.7          43      0
2        1.0  17.31500   36.5  13.4          56      0
3        1.0  21.04575   12.7   8.0          60      0
4        1.0  71.95631   60.9  67.5          30      0
0    0.144783
1    0.135095
2    0.200516
3    0.022899
4    0.408012
Name: CCP, dtype: float64


In [93]:
cross_elasticity = fun.cross_elasticity_1(logit_data, OLS)
cross_elasticity[:50]

Current coefficient: Intercept
Current model: I3_2013
Current model: Leaf_2013
Current model: Zoe_2013
Current model: Fortwo_2013
Current model: Model S_2013
Current model: up!_2013
Current coefficient: Price
Current model: I3_2013
Current model: Leaf_2013
Current model: Zoe_2013
Current model: Fortwo_2013
Current model: Model S_2013
Current model: up!_2013
Current coefficient: Range
Current model: I3_2013
Current model: Leaf_2013
Current model: Zoe_2013
Current model: Fortwo_2013
Current model: Model S_2013
Current model: up!_2013
Current coefficient: HP
Current model: I3_2013
Current model: Leaf_2013
Current model: Zoe_2013
Current model: Fortwo_2013
Current model: Model S_2013
Current model: up!_2013
Current coefficient: Chargetime
Current model: I3_2013
Current model: Leaf_2013
Current model: Zoe_2013
Current model: Fortwo_2013
Current model: Model S_2013
Current model: up!_2013
Current coefficient: China
Current model: I3_2013
Current model: Leaf_2013
Current model: Zoe_2013
Curre

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:161: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return cross_elasticity_table
/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:161: PerformanceWarning: indexing past lexsort depth may impact performance.
  return cross_elasticity_table


Cross_Elasticity
Model_year Model_year                              
I3_2013    I3_2013      Intercept          0.930089
                        Price              0.116607
                        Range             -0.356743
                        HP                -0.004015
                        Chargetime         0.034367
                        China                   0.0
           Leaf_2013    Intercept          0.867853
                        Price               0.11041
                        Range             -0.370108
                        HP                -0.003298
                        Chargetime         0.076606
                        China                   0.0
           Zoe_2013     Intercept          1.288115
                        Price               0.11185
                        Range             -0.611303
                        HP                -0.004462
                        Chargetime         0.148078
                        China                   0.0
           Fortwo_2013  Intercept          0.147104
                        Price              0.015526
                        Range             -0.024291
                        HP                -0.000304
                        Chargetime         0.018119
                        China                   0.0
           Model S_2013 Intercept          2.621077
                        Price               0.94582
                        Range              -2.07542
                        HP                -0.045733
                        Chargetime         0.161417
                        China                   0.0
           up!_2013     Intercept          0.569775
                        Price               0.04965
                        Range              -0.18965
                        HP                -0.001193
                        Chargetime         0.056143
                        China                   0.0
Leaf_2013  I3_2013      Intercept          0.930089
                        Price              0.116607
                        Range             -0.356743
                        HP                -0.004015
                        Chargetime         0.034367
                        China                   0.0
           Leaf_2013    Intercept          0.867853
                        Price               0.11041
                        Range             -0.370108
                        HP                -0.003298
                        Chargetime         0.076606
                        China                   0.0
           Zoe_2013     Intercept          1.288115
                        Price               0.11185

# Cost

In [94]:
logit_data['Cost']=fun.cost_original(logit_data, alpha)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost
0,10,2013,0.002237,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.144783,I3_2013,24.777491
1,132,2013,0.472036,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.135095,Leaf_2013,25.130534
2,158,2013,0.205817,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.200516,Zoe_2013,17.154336
3,167,2013,0.002237,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.022899,Fortwo_2013,19.638900
4,173,2013,0.250559,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.408012,Model S_2013,71.877353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.001277,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.004881,up!_2023,10.885023
330,188,2023,0.008354,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.016984,C40_2023,41.169712
331,189,2023,0.000065,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.021896,EX30_2023,35.353201
332,190,2023,0.031752,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.015324,XC40_2023,41.824344
